In [14]:
!pip install -Uqqq pycocotools
!pip install tqdm
!pip install opencv-python
!pip install scikit-image
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 10.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 10.0 MB/s eta 0:00:00


In [1]:
import cv2
import itertools
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from pycocotools.coco import COCO

from pathlib import Path
from PIL import Image

from itertools import groupby
from skimage import io
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import pycocotools.mask as mask_util

In [21]:
df = pd.read_csv('/home/eas/Enol/pycharm_projects/clipseg_steel_defect/Severstal/train.csv')
df.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [3]:
def rle_decode(mask_rle, shape):
    """
    Decodes run-length encoded segmentation mask string into 2d array

    Parameters
    ----------
    :param rle_mask (str): Run-length encoded segmentation mask string.
    :param shape (tuple): (height, width) of array to return
    :return mask [numpy.ndarray of shape (height, width)]: Decoded 2d segmentation mask
    """
    # Splits the RLE string into a list of string by whitespaces.
    s = mask_rle.split()

    # This creates two numpy arrays, one with the RLE starts and one with their respective lengths
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]

    # To obtain the end point we need to substract 1 to the length or start because the initial point counts.
    starts -= 1
    ends = starts + lengths

    # Create a 1D array of size H*W of zeros
    mask = np.zeros(shape[0]*shape[1], dtype=np.uint8)

    # Fill this array with ones in the positions where there is a mask using the RLE information
    for start, end in zip(starts, ends):
        mask[start:end] = 1

    # Reshape the 1D array into a 2D array so we can finally get the binary 2D mask.
    mask = mask.reshape(shape)
    return mask.T


In [31]:
import time
height = 1600
width = 256
read_path = '/home/eas/Enol/pycharm_projects/clipseg_steel_defect/Severstal/train_images/'
for idx in range(0, len(df)):
    mk = rle_decode(df.iloc[idx]['EncodedPixels'], (height, width))
    class_id = str(df.iloc[idx]['ClassId'])
    image_id = str(df.iloc[idx]['ImageId'])[0:9]
    contours, hierarchy = cv2.findContours(mk,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    file_name = read_path + image_id + '.jpg'
    img = cv2.imread(file_name)
    write_path = f'/home/eas/Enol/pycharm_projects/clipseg_steel_defect/Severstal/cropped_defects/{class_id}'
    output_name = f'{image_id}_{class_id}.jpg'
    if not os.path.exists(write_path):
        os.makedirs(write_path)
    for i, contour in enumerate(contours):
        bbox = cv2.boundingRect(contour)
        x = bbox[0]
        y = bbox[1]
        w = bbox[2]
        h = bbox[3]
        cropped_image = img[y:y+h, x:x+w, :]
        cv2.imwrite(f'{write_path}/{output_name}', cropped_image)
       
    